# Tech Intro

## Main tables

In [199]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [200]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = lead_list[4] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [201]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1119 17:13:09.817852 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1119 17:13:09.818751 20016 sched.cpp:241] Version: 1.11.0
I1119 17:13:09.819005 10543 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1119 17:13:09.819062 10543 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1119 17:13:09.820406 10546 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75413


## Read the spec-file from Hadoop and show some data sample

In [202]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-----+------------------+-----------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Show columns' properties

In [203]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- name: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- refer: string (nullable = true)
 |-- filial_id: string (nullable = true)
 |-- status: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- ag_object_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- url: string (nullable = true)
 |-- lead_id: string (nullable = true)
 |-- utm_campaign: string (nullable = true)
 |-- utm_content: string (nullable = true)
 |-- utm_source: string (nullable = true)
 |-- utm_medium: string (nullable = true)
 |-- utm_term: string (nullable = true)
 |-- gclid: string (nullable = true)
 |-- yclid: string (nullable = true)
 |-- source: string (nullable = true)



## Total rows (count)

In [204]:
eda_df.count()

13346

## Simple func to execute and show the result of the SQL-instruction

In [205]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [206]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [207]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|13346   |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [208]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,email,phone,name,comment,refer,filial_id,status,created_at,agent_id,...,url,lead_id,utm_campaign,utm_content,utm_source,utm_medium,utm_term,gclid,yclid,source
0,0,5125,1057,1590,1741,350,0,0,6,12498,...,5442,11382,13339,13318,8575,13334,13345,13346,12433,10134


In [209]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,email,phone,name,comment,refer,filial_id,status,created_at,agent_id,...,url,lead_id,utm_campaign,utm_content,utm_source,utm_medium,utm_term,gclid,yclid,source
0,0,0,1,3,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0


In [210]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,email,phone,name,comment,refer,filial_id,status,created_at,agent_id,...,url,lead_id,utm_campaign,utm_content,utm_source,utm_medium,utm_term,gclid,yclid,source
0,0,6399,464,6,1568,6320,0,0,0,0,...,0,0,0,0,0,0,0,0,42,0


In [211]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
email,5125,0,6399
phone,1057,1,464
name,1590,3,6
comment,1741,0,1568
refer,350,0,6320
filial_id,0,0,0
status,0,4,0
created_at,6,0,0
agent_id,12498,0,0


In [212]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [213]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
email,38.401,0.000,47.947
phone,7.920,0.007,3.477
name,11.914,0.022,0.045
comment,13.045,0.000,11.749
refer,2.623,0.000,47.355
filial_id,0.000,0.000,0.000
status,0.000,0.030,0.000
created_at,0.045,0.000,0.000
agent_id,93.646,0.000,0.000


### SQL-instructions

In [214]:
#Find dupes or misunderstanding or useless static values 13 346 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|13346             |
+------------------+

+---------------------+
|count(DISTINCT email)|
+---------------------+
|656                  |
+---------------------+

+---------------------+
|count(DISTINCT phone)|
+---------------------+
|7480                 |
+---------------------+

+--------------------+
|count(DISTINCT name)|
+--------------------+
|3172                |
+--------------------+

+-----------------------+
|count(DISTINCT comment)|
+-----------------------+
|4964                   |
+-----------------------+

+---------------------+
|count(DISTINCT refer)|
+---------------------+
|2239                 |
+---------------------+

+-------------------------+
|count(DISTINCT filial_id)|
+-------------------------+
|137                      |
+-------------------------+

+----------------------+
|count(DISTINCT status)|
+----------------------+
|4                     |
+----------------------+

+----------------

In [234]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [241]:
#Find dupes or misunderstanding or useless static values 13346 total

execute_template('lead_id')

+--------+
|lead_id |
+--------+
|22223672|
|22223670|
|22223669|
|22223469|
|22223467|
|22223462|
|22223455|
|22223453|
|22223448|
|22223434|
|22223433|
|22223429|
|22223394|
|22223393|
|22223389|
|22223313|
|22223311|
|22223300|
|22223274|
|22223273|
+--------+
only showing top 20 rows

None
+--------+
|lead_id |
+--------+
|null    |
|21715024|
|21715204|
|21715420|
|21715549|
|21715555|
|21716637|
|21716732|
|21717011|
|21717246|
|21717609|
|21717636|
|21717637|
|21718864|
|21719078|
|21719595|
|21720046|
|21720058|
|21720519|
|21721180|
+--------+
only showing top 20 rows



## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [242]:
spark.stop()

I1119 17:29:05.066896 10390 sched.cpp:2174] Asked to stop the driver
I1119 17:29:05.067034 10537 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75413
